# Timing for QobjEvo

Made by Eric Giguere

In [1]:
import qutip as qt
import numpy as np
from qutip import QobjEvo
%load_ext cython

In [2]:
N = 10
destroy, create, Id = qt.destroy(N), qt.create(N), qt.qeye(N)
def exp_i(t,args):
    return np.exp(-1j*t)
def cos_w(t,args):
    return np.cos(args["w"]*t)
tlist = np.linspace(0,10,10000)
tlistlog = np.logspace(-3,1,10000)

# state vector as np array
vec = np.arange(N)*.5+.5j
vec_super = np.arange(N**2)*.5+.5j
mat_c = (np.arange(N**2)*.5+.5j).reshape((10,10))
mat_f = np.asfortranarray(mat_c*1.)

# Construct QobjEvo of all type
td_cte1 = QobjEvo(Id)
td_cte2 = QobjEvo([Id])

td_func = QobjEvo([Id,[create,exp_i],[destroy,cos_w]],args={"w":2})
td_str = QobjEvo([Id,[create,"exp(-1j*t)"],[destroy,"cos(w*t)"]],args={'w':2.})
td_array = QobjEvo([Id,[create,np.exp(-1j*tlist)],[destroy,np.cos(2*tlist)]],tlist=tlist)
td_array_log = QobjEvo([Id,[create,np.exp(-1j*tlistlog)],[destroy,np.cos(2*tlistlog)]],tlist=tlistlog)

td_super = qt.liouvillian(td_func, c_ops=td_cte1)

## Timing: call

Timing check for all coeff formats.  
Once compiled, the cython method is called by the usual method.

In [3]:
# The compiled version build the scipy object from the sparce matrix if not used with data=True
print("Time for cte")
td_cte1.compiled = False
%timeit td_cte1(2)
%timeit td_cte1(2, data=True)
print("Time for compiled cte")
td_cte1.compile()
%timeit td_cte1(2)
%timeit td_cte1(2, data=True)

Time for cte
53.7 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
11.1 µs ± 72.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Time for compiled cte
53.5 µs ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
11.1 µs ± 39.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
print("Time for str")
td_str.compiled = False
%timeit td_str(2)
%timeit td_str(2, data=True)
td_str.compile()
%timeit td_str(2)
%timeit td_str(2, data=True)

Time for str
604 µs ± 19.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
195 µs ± 2.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
70.3 µs ± 82.4 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
13.2 µs ± 44.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
print("Time for function")
td_func.compiled = False
%timeit td_func(2)
%timeit td_func(2, data=True)
td_func.compile()
%timeit td_func(2)
%timeit td_func(2, data=True)

Time for function
534 µs ± 6.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
143 µs ± 321 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
78.8 µs ± 912 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
20.5 µs ± 302 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
print("Time for array")
td_array.compiled = False
%timeit td_array(2)
%timeit td_array(2, data=True)
td_array.compile()
%timeit td_array(2)
%timeit td_array(2, data=True)

Time for array
565 µs ± 7.66 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
180 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
66.8 µs ± 866 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
12.1 µs ± 337 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [7]:
print("Time for array with variable time step")
td_array_log.compiled = False
%timeit td_array_log(2)
%timeit td_array_log(2, data=True)
td_array_log.compile()
%timeit td_array_log(2)
%timeit td_array_log(2, data=True)

Time for array with variable time step
566 µs ± 32.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
183 µs ± 983 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
68.3 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
12.6 µs ± 28.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing: multiplication with a vector
The QobjEvo at t x vec, used by most solver

In [8]:
print("timing for rhs: cte")
td_cte1.compiled = False
%timeit qt.cy.spmv(td_cte1(2, data=True), vec)
%timeit qt.cy.spmv(Id.data, vec)
%timeit td_cte1.mul_vec(2,vec)
td_cte1.compile()
%timeit td_cte1.mul_vec(2,vec)

timing for rhs: cte
19.5 µs ± 416 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
5.45 µs ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
20.9 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
5.91 µs ± 13 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
print("timing for rhs: str")
td_str.compiled = False
%timeit qt.cy.spmv(td_str(2, data=True), vec)
%timeit td_str.mul_vec(2,vec)
td_str.compile()
%timeit td_str.mul_vec(2,vec)

timing for rhs: str
212 µs ± 7.66 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
218 µs ± 17.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
6.77 µs ± 19.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
print("timing for rhs: cte")
td_func.compiled = False
%timeit qt.cy.spmv(td_func(2, data=True), vec)
%timeit td_func.mul_vec(2,vec)
td_func.compile()
%timeit td_func.mul_vec(2,vec)

timing for rhs: cte
154 µs ± 247 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
156 µs ± 2.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
13.9 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
print("timing for rhs: cte")
td_array.compiled = False
%timeit qt.cy.spmv(td_array(2, data=True), vec)
%timeit td_array.mul_vec(2,vec)
td_array.compile()
%timeit td_array.mul_vec(2,vec)

timing for rhs: cte
197 µs ± 790 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
199 µs ± 3.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
6.45 µs ± 46.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
print("timing for rhs: cte")
td_array_log.compiled = False
%timeit qt.cy.spmv(td_array_log(2, data=True), vec)
%timeit td_array_log.mul_vec(2,vec)
td_array_log.compile()
%timeit td_array_log.mul_vec(2,vec)

timing for rhs: cte
202 µs ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
210 µs ± 23.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
6.59 µs ± 98.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing: multiplication with matrix
QobjEvo at t time same shape matrix, used for operator evolution

In [13]:
print("timing for mul_mat: c format sparce")
td_str.compiled = False
%timeit td_str(2, data=True) * mat_c
%timeit td_str.mul_mat(2,mat_c)
td_str.compile()
%timeit td_str.mul_mat(2,mat_c)

timing for mul_mat: c format sparce
205 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
207 µs ± 2.99 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
8.64 µs ± 70.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
print("timing for mul_mat: c format dense")
td_str.compiled = False
%timeit td_str(2, data=True) * mat_c
%timeit td_str.mul_mat(2,mat_c)
td_str.compile(dense=True)
%timeit td_str.mul_mat(2,mat_c)

timing for mul_mat: c format dense
208 µs ± 9.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
218 µs ± 22.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
12 µs ± 248 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [15]:
print("timing for mul_mat: f format sparce")
td_str.compiled = False
%timeit td_str(2, data=True) * mat_f
%timeit td_str.mul_mat(2,mat_f)
td_str.compile()
%timeit td_str.mul_mat(2,mat_f)

timing for mul_mat: f format sparce
209 µs ± 5.56 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
216 µs ± 7.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
10.5 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [16]:
print("timing for mul_mat: f format dense")
td_str.compiled = False
%timeit td_str(2, data=True) * mat_f
%timeit td_str.mul_mat(2,mat_f)
td_str.compile(dense=True)
%timeit td_str.mul_mat(2,mat_f)

timing for mul_mat: f format dense
205 µs ± 5.33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
208 µs ± 6.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
12.2 µs ± 340 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing: expect
expectation value of QobjEvo(t) and psi/rho

In [17]:
print("timing for expect: cte")
td_cte1.compiled = False
%timeit qt.cy.cy_expect_psi(td_cte1(2, data=True), vec, 0)
%timeit td_cte1.expect(2,vec,0)
td_cte1.compile()
%timeit td_cte1.expect(2,vec,0)

timing for expect: cte
16.1 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
18.2 µs ± 239 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.25 µs ± 111 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [18]:
print("timing for expect: str")
td_str.compiled = False
%timeit qt.cy.cy_expect_psi(td_str(2, data=True), vec, 0)
%timeit td_str.expect(2,vec,0)
td_str.compile()
%timeit td_str.expect(2,vec,0)

timing for expect: str
198 µs ± 5.15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
199 µs ± 5.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.12 µs ± 52.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [19]:
print("timing for expect: func")
td_func.compiled = False
%timeit qt.cy.cy_expect_psi(td_func(2, data=True), vec, 0)
%timeit td_func.expect(2,vec,0)
td_func.compile()
%timeit td_func.expect(2,vec,0)

timing for expect: func
153 µs ± 3.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
155 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
14.8 µs ± 150 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
print("timing for expect: array")
td_array.compiled = False
%timeit qt.cy.cy_expect_psi(td_array(2, data=True), vec, 0)
%timeit td_array.expect(2,vec,0)
td_array.compile()
%timeit td_array.expect(2,vec,0)

timing for expect: array
206 µs ± 9.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
206 µs ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.22 µs ± 237 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
print("timing for expect: array logscale")
td_array_log.compiled = False
%timeit qt.cy.cy_expect_psi(td_array_log(2, data=True), vec, 0)
%timeit td_array_log.expect(2,vec,0)
td_array_log.compile()
%timeit td_array_log.expect(2,vec,0)

timing for expect: array logscale
201 µs ± 9.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
196 µs ± 3.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.21 µs ± 84.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [22]:
print("timing for expect_rho: func")
td_super.compiled = False
%timeit qt.cy.cy_expect_rho_vec(td_super(2, data=True), vec_super, 0)
%timeit td_super.expect(2,vec_super,0)
td_super.compile()
%timeit td_super.expect(2,vec_super,0)

timing for expect_rho: func
165 µs ± 2.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
166 µs ± 608 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
10.6 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing: call with other arguments

In [23]:
td_args = QobjEvo([Id,[destroy,cos_w]],args={"w":1})

In [24]:
print("timing for function")
%timeit td_args(2, args={"w":2})
%timeit td_args(2)
%timeit td_args(2, args={"w":2}, data=True)
%timeit td_args(2, data=True)
td_args.compile()
%timeit td_args(2, args={"w":2})
%timeit td_args(2)
%timeit td_args(2, args={"w":2}, data=True)
%timeit td_args(2, data=True)

timing for function


TypeError: __call__() got an unexpected keyword argument 'args'

In [ ]:
td_args = QobjEvo([Id,[destroy,"cos(w*t)"]],args={"w":1})

In [ ]:
print("timing for string")
%timeit td_args(2, args={"w":2})
%timeit td_args(2)
%timeit td_args(2, args={"w":2}, data=True)
%timeit td_args(2, data=True)
td_args.compile()
%timeit td_args(2, args={"w":2})
%timeit td_args(2)
%timeit td_args(2, args={"w":2}, data=True)
%timeit td_args(2, data=True)

## Timing: call that depend on state


In [ ]:
def coeff_state(t, args):
    return np.max(args["vec"])*args["w"]
td_state = QobjEvo([Id, [destroy, coeff_state]],args={"w":1,"vec=vec":vec})
%timeit td_state(2, state=vec, data=True)
td_state.compile()
%timeit td_state(2, state=vec, data=True)

In [ ]:
def coeff_state(t, args):
    return args["e1"]
td_state = QobjEvo([Id, [destroy, coeff_state]], args={"e1=expect":qt.qeye(N)})
%timeit td_state(2, state=vec, data=True)
td_state.compile()
%timeit td_state(2, state=vec, data=True)

In [ ]:
td_state = QobjEvo([Id, [destroy, "e1"]], args={"e1=expect":qt.qeye(N)})
%timeit td_state(2, state=vec, data=True)
td_state.compile()
%timeit td_state(2, state=vec, data=True)

In [ ]:
qt.about()